# 🔧 QAOA Codelab: Quantum Approximate Optimization Algorithm

## From MaxCut to Quantum Circuits

| Property | Value |
|----------|-------|
| **Algorithm** | QAOA (Quantum Approximate Optimization Algorithm) |
| **Difficulty** | 🔴 Advanced |
| **Time** | 90-120 minutes |
| **Prerequisites** | Module-10-QAOA-Advanced.md, QPE basics |
| **Qiskit Version** | 2.x |

---

## 🎯 Learning Objectives

By completing this codelab, you will:

1. ✅ Implement ZZ gates for cost Hamiltonian evolution
2. ✅ Build complete QAOA circuits for MaxCut
3. ✅ Compute expectation values from measurement results
4. ✅ Optimize QAOA parameters using classical optimizers
5. ✅ Analyze solution quality vs circuit depth

---

## Section 1: Environment Setup

In [ ]:
# Core imports
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter

# Qiskit imports
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit.primitives import StatevectorSampler, SamplerResult
from qiskit_ibm_runtime.fake_provider import FakeAlmadenV2
from qiskit.visualization import plot_histogram

# Graph library
%pip install networkx
import networkx as nx

# Optimization
from scipy.optimize import minimize
# Initialize fake backend and sampler
fake_backend = FakeAlmadenV2()
sampler = StatevectorSampler()

# Version check
import qiskit
print(f"Qiskit version: {qiskit.__version__}")
assert int(qiskit.__version__.split('.')[0]) >= 1, "Requires Qiskit 1.x or 2.x"
plt.rcParams['font.size'] = 12
# Configure plotting

plt.rcParams['figure.figsize'] = (10, 6)
print("✅ All imports successful!")

plt.rcParams['font.size'] = 12

---

## Section 2: Theory Recap

### The MaxCut Problem

Given a graph $G = (V, E)$, partition vertices into two sets to maximize edges cut.

### QAOA Structure

$$|\psi(\gamma, \beta)\rangle = e^{-i\beta_p H_M} e^{-i\gamma_p H_C} \cdots e^{-i\beta_1 H_M} e^{-i\gamma_1 H_C} |+\rangle^{\otimes n}$$

Where:
- **Cost Hamiltonian**: $H_C = \sum_{(i,j) \in E} \frac{1 - Z_i Z_j}{2}$
- **Mixer Hamiltonian**: $H_M = \sum_i X_i$

### ZZ Gate Implementation

```
q_i: ───●───────────────●───
        │               │
q_j: ───X───Rz(γ)───────X───
```

This implements $e^{i\gamma Z_i Z_j / 2}$

---

## Section 3: Create Test Graphs

In [ ]:
def create_sample_graphs():
    """
    Create several test graphs for QAOA experiments.
    
    Returns:
        dict: Collection of named graphs
    """
    graphs = {}
    
    # Simple triangle (3 nodes)
    graphs['triangle'] = nx.Graph()
    graphs['triangle'].add_edges_from([(0, 1), (1, 2), (0, 2)])
    
    # Square (4 nodes, ring)
    graphs['square'] = nx.Graph()
    graphs['square'].add_edges_from([(0, 1), (1, 2), (2, 3), (3, 0)])
    
    # Complete graph K4 (4 nodes, all connected)
    graphs['K4'] = nx.complete_graph(4)
    
    # Butterfly graph (5 nodes)
    graphs['butterfly'] = nx.Graph()
    graphs['butterfly'].add_edges_from([
        (0, 1), (0, 2), (1, 2),  # Left triangle
        (2, 3), (2, 4), (3, 4)   # Right triangle
    ])
    
    return graphs

# Create and visualize
graphs = create_sample_graphs()

fig, axes = plt.subplots(1, 4, figsize=(16, 4))
for ax, (name, G) in zip(axes, graphs.items()):
    pos = nx.spring_layout(G, seed=42)
    nx.draw(G, pos, ax=ax, with_labels=True, node_color='lightblue',
            node_size=500, font_size=14, font_weight='bold', edge_color='gray')
    ax.set_title(f"{name}\n{G.number_of_nodes()} nodes, {G.number_of_edges()} edges")
plt.tight_layout()
plt.show()

print("\n📊 Graph Statistics:")
for name, G in graphs.items():
    max_cut = G.number_of_edges()  # Upper bound for MaxCut
    print(f"  {name}: {G.number_of_nodes()} nodes, {G.number_of_edges()} edges, max possible cut ≤ {max_cut}")

---

## Section 4: ZZ Gate Implementation

In [ ]:
def zz_gate(qc: QuantumCircuit, q1: int, q2: int, gamma: float) -> None:
    """
    Apply ZZ evolution gate: exp(i * gamma * Z_i Z_j / 2)
    
    This is implemented as a CNOT sandwich:
    - CNOT maps ZZ eigenvalue to Z on target qubit
    - Rz applies phase based on that eigenvalue
    - CNOT undoes the mapping
    
    Parameters:
        qc: QuantumCircuit to modify
        q1: First qubit index
        q2: Second qubit index  
        gamma: Evolution angle
    """
    qc.cx(q1, q2)      # Map ZZ eigenvalue to Z on q2
    qc.rz(gamma, q2)   # Apply phase based on eigenvalue
    qc.cx(q1, q2)      # Undo the mapping

# Demonstrate ZZ gate
demo_qc = QuantumCircuit(2)
demo_qc.h([0, 1])  # Create superposition
zz_gate(demo_qc, 0, 1, np.pi/4)

print("ZZ Gate Circuit:")
print(demo_qc.draw(output='text'))

# Verify unitarity by checking matrix
from qiskit.quantum_info import Operator
U = Operator(demo_qc).data
print(f"\n✅ Circuit is unitary: {np.allclose(U @ U.conj().T, np.eye(4))}")

---

## Section 5: QAOA Layer Construction

In [ ]:
def cost_layer(qc: QuantumCircuit, graph: nx.Graph, gamma: float) -> None:
    """
    Apply cost Hamiltonian evolution: exp(-i * gamma * H_C)
    
    For MaxCut, H_C = sum_{(i,j) in E} (1 - Z_i Z_j) / 2
    We implement ZZ gates for each edge.
    
    Parameters:
        qc: QuantumCircuit to modify
        graph: NetworkX graph
        gamma: Cost evolution angle
    """
    for (i, j) in graph.edges():
        zz_gate(qc, i, j, gamma)

def mixer_layer(qc: QuantumCircuit, n_qubits: int, beta: float) -> None:
    """
    Apply mixer Hamiltonian evolution: exp(-i * beta * H_M)
    
    For standard QAOA, H_M = sum_i X_i
    This factorizes into individual Rx rotations.
    
    Parameters:
        qc: QuantumCircuit to modify
        n_qubits: Number of qubits
        beta: Mixer evolution angle
    """
    for i in range(n_qubits):
        qc.rx(2 * beta, i)

def qaoa_layer(qc: QuantumCircuit, graph: nx.Graph, gamma: float, beta: float) -> None:
    """
    Apply one complete QAOA layer: cost evolution followed by mixer.
    
    Parameters:
        qc: QuantumCircuit to modify
        graph: NetworkX graph
        gamma: Cost evolution angle
        beta: Mixer evolution angle
    """
    cost_layer(qc, graph, gamma)
    qc.barrier()  # Visual separation
    mixer_layer(qc, graph.number_of_nodes(), beta)

# Demonstrate one QAOA layer on square graph
square = graphs['square']
layer_demo = QuantumCircuit(4)
layer_demo.h(range(4))  # Initial superposition
layer_demo.barrier()
qaoa_layer(layer_demo, square, gamma=0.5, beta=0.3)

print("Single QAOA Layer (Square Graph):")
print(layer_demo.draw(output='text', fold=80))

---

## Section 5.5: State Evolution Analysis

### 5.5.1 The "Adiabatic-Inspired Alternation" Rule

**Key Identity**: The QAOA ansatz state is built by alternating applications of cost and mixer evolution:

$$|\psi(\gamma, \beta)\rangle = \prod_{l=1}^{p} e^{-i\beta_l H_M} e^{-i\gamma_l H_C} |+\rangle^{\otimes n}$$

**Why This Works** (Connection to Adiabatic Computing):

1. **Initial State**: $|+\rangle^{\otimes n}$ is the ground state of mixer Hamiltonian $H_M = \sum_i X_i$
2. **Target State**: Solution to MaxCut is ground state of cost Hamiltonian $H_C$  
3. **Alternation**: Interleaving $e^{-i\gamma H_C}$ and $e^{-i\beta H_M}$ mimics slow adiabatic evolution
4. **Key Insight**: As $p \to \infty$, optimal QAOA reproduces adiabatic evolution (Trotterization)

**The "ZZ Sandwich" for Cost Evolution**:

For an edge $(i,j)$, the cost term $e^{i\gamma Z_i Z_j}$ is implemented as:

$$\text{CNOT}_{i,j} \cdot R_Z(\gamma)_j \cdot \text{CNOT}_{i,j}$$

This works because:
- CNOT maps $|ab\rangle \to |a, a\oplus b\rangle$
- $R_Z$ applies phase based on XOR parity
- Second CNOT restores original state with accumulated phase

### 5.5.2 State Evolution at Each Stage

For a simple 3-node triangle graph with p=1 QAOA:

| Stage | State Description | Mathematical Form |
|-------|-------------------|-------------------|
| **Initial** | All qubits in $\|0\rangle$ | $\|000\rangle$ |
| **After Hadamard** | Uniform superposition (ground state of $H_M$) | $\|+\rangle^{\otimes 3} = \frac{1}{\sqrt{8}}\sum_{x \in \{0,1\}^3} \|x\rangle$ |
| **After Cost Layer** | Phase rotation based on cut values | $\frac{1}{\sqrt{8}}\sum_x e^{i\gamma \cdot C(x)} \|x\rangle$ |
| **After Mixer Layer** | Amplitudes redistributed by $R_X$ rotations | Complex superposition favoring good solutions |
| **After Measurement** | Classical bitstring | Most likely: high-cut configurations |

**Key Insight**: "The cost layer imprints the problem structure as phases. The mixer layer converts these phase differences into amplitude differences through interference."

**What each layer does**:
- **Cost layer** ($e^{-i\gamma H_C}$): States with more edges cut accumulate different phases
- **Mixer layer** ($e^{-i\beta H_M}$): Single-qubit $R_X(2\beta)$ rotations create interference

In [ ]:
def count_edges_cut(bitstring: str, graph: nx.Graph) -> int:
    """
    Count the number of edges cut by a given bitstring partition.
    
    Args:
        bitstring: Binary string representing partition (e.g., "0101")
        graph: NetworkX graph
    
    Returns:
        int: Number of edges cut
    """
    cut_count = 0
    for u, v in graph.edges():
        if bitstring[u] != bitstring[v]:
            cut_count += 1
    return cut_count

def trace_qaoa_evolution(graph: nx.Graph, gamma: float, beta: float, 
                          show_amplitudes: bool = True) -> dict:
    """
    Trace QAOA state evolution through each stage.
    
    This function demonstrates the "Adiabatic-Inspired Alternation" rule
    by showing how the state transforms at each step.
    
    Args:
        graph: NetworkX graph defining the problem
        gamma: Cost evolution parameter
        beta: Mixer evolution parameter
        show_amplitudes: Whether to print amplitude details
    
    Returns:
        dict: State information at each stage
    """
    from qiskit.quantum_info import Statevector
    
    n_qubits = graph.number_of_nodes()
    stages = {}
    
    # Stage 1: Initial state |0...0⟩
    qc = QuantumCircuit(n_qubits)
    sv = Statevector(qc)
    stages['initial'] = {
        'description': '|0⟩^⊗n',
        'statevector': sv,
        'max_amplitude': np.max(np.abs(sv.data))
    }
    
    # Stage 2: After Hadamard (uniform superposition)
    qc.h(range(n_qubits))
    sv = Statevector(qc)
    stages['after_hadamard'] = {
        'description': '|+⟩^⊗n (uniform superposition)',
        'statevector': sv,
        'max_amplitude': np.max(np.abs(sv.data)),
        'uniform_amplitude': 1/np.sqrt(2**n_qubits)
    }
    
    # Stage 3: After Cost Layer
    cost_layer(qc, graph, gamma)
    sv = Statevector(qc)
    
    # Calculate phases imprinted by cost function
    phases = {}
    for i in range(2**n_qubits):
        bitstring = format(i, f'0{n_qubits}b')
        cut_value = count_edges_cut(bitstring, graph)
        phases[bitstring] = cut_value
    
    stages['after_cost'] = {
        'description': f'Phases imprinted by H_C (γ={gamma:.3f})',
        'statevector': sv,
        'max_amplitude': np.max(np.abs(sv.data)),
        'cut_values': phases
    }
    
    # Stage 4: After Mixer Layer  
    mixer_layer(qc, n_qubits, beta)
    sv = Statevector(qc)
    
    # Find amplitudes for each cut value
    cut_to_amplitude = {}
    for i in range(2**n_qubits):
        bitstring = format(i, f'0{n_qubits}b')
        cut_value = count_edges_cut(bitstring, graph)
        if cut_value not in cut_to_amplitude:
            cut_to_amplitude[cut_value] = []
        cut_to_amplitude[cut_value].append((bitstring, np.abs(sv.data[i])**2))
    
    stages['after_mixer'] = {
        'description': f'After mixer rotation (β={beta:.3f})',
        'statevector': sv,
        'max_amplitude': np.max(np.abs(sv.data)),
        'prob_by_cut': {k: sum(p for _, p in v) for k, v in cut_to_amplitude.items()}
    }
    
    # Print detailed trace
    print("="*70)
    print("QAOA STATE EVOLUTION TRACE")
    print("="*70)
    print(f"Graph: {n_qubits} nodes, {graph.number_of_edges()} edges")
    print(f"Parameters: γ = {gamma:.4f}, β = {beta:.4f}")
    print("-"*70)
    
    print("\n📍 Stage 1: Initial State")
    print(f"   {stages['initial']['description']}")
    print(f"   Only |{'0'*n_qubits}⟩ has amplitude 1.0")
    
    print("\n📍 Stage 2: After Hadamard Gates")
    print(f"   {stages['after_hadamard']['description']}")
    print(f"   All {2**n_qubits} states have amplitude {stages['after_hadamard']['uniform_amplitude']:.4f}")
    print(f"   This is the GROUND STATE of mixer Hamiltonian H_M = ΣXᵢ")
    
    print("\n📍 Stage 3: After Cost Layer e^{-iγH_C}")
    print(f"   {stages['after_cost']['description']}")
    print("   Phase imprinted based on cut value:")
    if show_amplitudes:
        for bs, cut in sorted(stages['after_cost']['cut_values'].items(), 
                              key=lambda x: -x[1]):
            phase = gamma * cut  # Simplified; actual phase is more complex
            print(f"      |{bs}⟩: cut={cut} edges → phase ∝ {cut}γ")
    
    print("\n📍 Stage 4: After Mixer Layer e^{-iβH_M}")
    print(f"   {stages['after_mixer']['description']}")
    print("   Probability distribution by cut value:")
    for cut in sorted(stages['after_mixer']['prob_by_cut'].keys(), reverse=True):
        prob = stages['after_mixer']['prob_by_cut'][cut]
        bar = '█' * int(prob * 40)
        print(f"      Cut={cut}: {prob:.3f} {bar}")
    
    return stages

# Trace evolution on triangle graph
triangle = graphs['triangle']
print("\n🔍 Tracing QAOA evolution on triangle graph (3 edges, max cut = 2):\n")
stages = trace_qaoa_evolution(triangle, gamma=0.5, beta=0.4)

### 5.5.3 The ZZ Gate: Heart of the Cost Layer

The ZZ interaction for each edge is the key to encoding the MaxCut problem.

**Mathematical Identity**:
$$e^{i\gamma Z_i Z_j} = \text{CNOT}_{i,j} \cdot R_Z(\gamma)_j \cdot \text{CNOT}_{i,j}$$

Let's verify this "CNOT sandwich" implements the correct phases:

In [ ]:
def verify_zz_gate(gamma: float = np.pi/4):
    """
    Verify that the CNOT-Rz-CNOT sandwich implements ZZ evolution.
    
    The key insight from the lecture:
    - ZZ|ab⟩ = (-1)^(a XOR b) |ab⟩  (eigenvalue +1 if same, -1 if different)
    - When a=b (same partition): ZZ eigenvalue = +1 → phase = +γ
    - When a≠b (different partition = edge CUT): ZZ eigenvalue = -1 → phase = -γ
    
    This is exactly what we need for MaxCut!
    """
    from qiskit.quantum_info import Statevector
    
    print("="*60)
    print("VERIFYING ZZ GATE IMPLEMENTATION")
    print("="*60)
    print(f"Testing with γ = {gamma:.4f} = π/{np.pi/gamma:.1f}")
    print()
    
    basis_states = ['00', '01', '10', '11']
    
    print("The ZZ gate eigenvalue structure:")
    print("-"*60)
    print("|ab⟩  | a⊕b | Z_a·Z_b | Expected Phase | MaxCut Meaning")
    print("-"*60)
    
    results = []
    for bs in basis_states:
        # Prepare basis state
        qc = QuantumCircuit(2)
        if bs[1] == '1':  # qubit 0 (rightmost)
            qc.x(0)
        if bs[0] == '1':  # qubit 1
            qc.x(1)
        
        # Get initial phase
        sv_before = Statevector(qc)
        
        # Apply ZZ gate
        zz_gate(qc, 0, 1, gamma)
        sv_after = Statevector(qc)
        
        # Calculate phase change
        idx = int(bs, 2)
        phase_before = np.angle(sv_before.data[idx])
        phase_after = np.angle(sv_after.data[idx])
        phase_change = phase_after - phase_before
        
        # Calculate expected
        a, b = int(bs[0]), int(bs[1])
        xor = a ^ b
        zz_eigenvalue = (-1) ** xor
        expected_phase = gamma * zz_eigenvalue / 2  # Factor of 2 from Rz definition
        
        # MaxCut interpretation
        if a == b:
            cut_status = "Same side (not cut)"
        else:
            cut_status = "Different sides (CUT!)"
        
        print(f"|{bs}⟩  |  {xor}  |   {zz_eigenvalue:+d}    |   {expected_phase:+.4f}   | {cut_status}")
        results.append((bs, phase_change, expected_phase))
    
    print("-"*60)
    print("\n✅ Key Insight: States with endpoints in DIFFERENT partitions")
    print("   (edges that ARE cut) get NEGATIVE phase contribution.")
    print("   This is how H_C 'knows' the cut value of each configuration!")
    
    return results

# Verify the ZZ gate
zz_results = verify_zz_gate(gamma=np.pi/4)

### 5.5.4 Interactive Parameter Landscape Explorer

The QAOA energy landscape for p=1 is particularly interesting because optimal parameters can often be found analytically. Let's visualize how different (γ, β) values affect the probability of finding good solutions:

In [ ]:
def create_qaoa_circuit(graph: nx.Graph, gammas: list, betas: list, measure: bool = True) -> QuantumCircuit:
    """
    Create a complete QAOA circuit with p layers.
    
    Args:
        graph: NetworkX graph defining the problem
        gammas: List of gamma parameters (length p)
        betas: List of beta parameters (length p)
        measure: Whether to add measurement gates
    
    Returns:
        QuantumCircuit: Complete QAOA circuit
    """
    n_qubits = graph.number_of_nodes()
    p = len(gammas)
    
    # Initialize circuit
    if measure:
        qc = QuantumCircuit(n_qubits, n_qubits)
    else:
        qc = QuantumCircuit(n_qubits)
    
    # Initial state preparation: uniform superposition
    qc.h(range(n_qubits))
    qc.barrier()
    
    # Apply p QAOA layers
    for i in range(p):
        qaoa_layer(qc, graph, gammas[i], betas[i])
        qc.barrier()
    
    # Add measurements if requested
    if measure:
        qc.measure(range(n_qubits), range(n_qubits))
    
    return qc

def explore_qaoa_landscape(graph: nx.Graph, n_points: int = 20, shots: int = 256):
    """
    Visualize the QAOA parameter landscape for p=1.
    
    Shows how expected cut value varies with (γ, β) parameters.
    Helps build intuition for why certain parameter ranges work better.
    
    Args:
        graph: NetworkX graph
        n_points: Resolution of the grid
        shots: Shots per circuit evaluation
    """
    from qiskit.quantum_info import Statevector
    
    # Create parameter grid
    gammas = np.linspace(0, np.pi, n_points)
    betas = np.linspace(0, np.pi/2, n_points)
    Gamma, Beta = np.meshgrid(gammas, betas)
    
    # Evaluate expected cut at each point (using statevector for speed)
    expected_cuts = np.zeros_like(Gamma)
    
    print(f"Computing QAOA landscape for {graph.number_of_nodes()}-node graph...")
    
    for i, gamma in enumerate(gammas):
        for j, beta in enumerate(betas):
            # Build circuit without measurement
            qc = create_qaoa_circuit(graph, [gamma], [beta], measure=False)
            sv = Statevector(qc)
            probs = np.abs(sv.data)**2
            
            # Calculate expected cuts
            total_cut = 0
            for k in range(len(probs)):
                bitstring = format(k, f'0{graph.number_of_nodes()}b')
                cut = count_edges_cut(bitstring, graph)
                total_cut += probs[k] * cut
            
            expected_cuts[j, i] = total_cut
    
    # Find optimal parameters
    max_idx = np.unravel_index(np.argmax(expected_cuts), expected_cuts.shape)
    optimal_gamma = gammas[max_idx[1]]
    optimal_beta = betas[max_idx[0]]
    max_expected = expected_cuts[max_idx]
    
    # Visualize
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Heatmap
    ax1 = axes[0]
    im = ax1.contourf(Gamma, Beta, expected_cuts, levels=20, cmap='viridis')
    ax1.plot(optimal_gamma, optimal_beta, 'r*', markersize=20, label=f'Optimal')
    ax1.set_xlabel('γ (cost parameter)', fontsize=12)
    ax1.set_ylabel('β (mixer parameter)', fontsize=12)
    ax1.set_title(f'QAOA p=1 Landscape\nMax = {max_expected:.3f} at γ={optimal_gamma:.2f}, β={optimal_beta:.2f}')
    plt.colorbar(im, ax=ax1, label='Expected Cuts')
    ax1.legend()
    
    # Cross-sections
    ax2 = axes[1]
    mid_beta_idx = n_points // 4  # β ≈ π/8
    mid_gamma_idx = n_points // 2  # γ ≈ π/2
    
    ax2.plot(gammas, expected_cuts[mid_beta_idx, :], 'b-', linewidth=2, 
             label=f'β = {betas[mid_beta_idx]:.2f} (vary γ)')
    ax2.plot(betas, expected_cuts[:, mid_gamma_idx], 'r--', linewidth=2,
             label=f'γ = {gammas[mid_gamma_idx]:.2f} (vary β)')
    ax2.axhline(y=graph.number_of_edges(), color='g', linestyle=':', 
                label=f'Max possible = {graph.number_of_edges()}')
    ax2.set_xlabel('Parameter value', fontsize=12)
    ax2.set_ylabel('Expected Cuts', fontsize=12)
    ax2.set_title('Parameter Cross-Sections')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print(f"\n📊 Landscape Analysis:")
    print(f"   Optimal γ* = {optimal_gamma:.4f} (≈ {optimal_gamma/np.pi:.2f}π)")
    print(f"   Optimal β* = {optimal_beta:.4f} (≈ {optimal_beta/np.pi:.2f}π)")
    print(f"   Expected cuts at optimum: {max_expected:.3f}")
    print(f"   Maximum possible cuts: {graph.number_of_edges()}")
    print(f"   Approximation ratio: {max_expected/graph.number_of_edges():.1%}")
    
    return {'optimal_gamma': optimal_gamma, 'optimal_beta': optimal_beta, 
            'max_expected': max_expected, 'landscape': expected_cuts}

# Explore landscape for square graph
square = graphs['square']
landscape_result = explore_qaoa_landscape(square, n_points=25)

### 5.5.5 From Cost Function to Ising Hamiltonian

The lecture emphasizes the key transformation: converting the classical cost function to a quantum Hamiltonian.

**MaxCut Cost Function**:
$$C(x) = \sum_{(i,j) \in E} x_i(1-x_j) + (1-x_i)x_j = \sum_{(i,j) \in E} \frac{1 - (-1)^{x_i}(-1)^{x_j}}{2}$$

**Quantum Hamiltonian** (replacing $(-1)^{x_i}$ with $Z_i$):
$$H_C = \sum_{(i,j) \in E} \frac{1 - Z_i Z_j}{2}$$

This is an **Ising model** Hamiltonian! The eigenvalue on bitstring $|x\rangle$ equals $C(x)$.

In [ ]:
def demonstrate_ising_mapping(graph: nx.Graph):
    """
    Demonstrate the mapping from MaxCut cost function to Ising Hamiltonian.
    
    Key insight from lecture:
    - Classical: x_i ∈ {0, 1} (bit values)
    - Quantum: Z_i eigenvalue ∈ {+1, -1} where Z_i|x_i⟩ = (-1)^{x_i}|x_i⟩
    - The mapping x_i → (1 - Z_i)/2 converts bits to spins
    """
    from qiskit.quantum_info import Operator
    
    n = graph.number_of_nodes()
    
    print("="*70)
    print("ISING HAMILTONIAN MAPPING FOR MAXCUT")
    print("="*70)
    print(f"Graph: {n} nodes, {graph.number_of_edges()} edges")
    print(f"Edges: {list(graph.edges())}")
    print()
    
    # Build Hamiltonian matrix explicitly
    H = np.zeros((2**n, 2**n))
    
    # Z operator
    Z = np.array([[1, 0], [0, -1]])
    I = np.eye(2)
    
    print("Cost Hamiltonian H_C = Σ_{(i,j)∈E} (1 - Z_i⊗Z_j)/2")
    print("-"*70)
    
    for (i, j) in graph.edges():
        # Build Z_i ⊗ Z_j term
        ops = [I] * n
        ops[i] = Z
        ops[j] = Z
        
        ZZ = ops[0]
        for op in ops[1:]:
            ZZ = np.kron(ZZ, op)
        
        # Add (I - ZZ)/2 to Hamiltonian
        H += (np.eye(2**n) - ZZ) / 2
        print(f"   Edge ({i},{j}): Added (I - Z_{i}⊗Z_{j})/2")
    
    print("-"*70)
    print("\nVerifying eigenvalues match cut values:")
    print()
    print("| Bitstring | Classical C(x) | Quantum ⟨x|H_C|x⟩ | Match? |")
    print("|-----------|----------------|-------------------|--------|")
    
    all_match = True
    for k in range(2**n):
        bitstring = format(k, f'0{n}b')
        
        # Classical cost
        classical_cost = count_edges_cut(bitstring, graph)
        
        # Quantum eigenvalue
        state_vector = np.zeros(2**n)
        state_vector[k] = 1
        quantum_cost = state_vector @ H @ state_vector
        
        match = np.isclose(classical_cost, quantum_cost)
        all_match = all_match and match
        
        status = "✅" if match else "❌"
        print(f"| {bitstring}      | {classical_cost:^14.1f} | {quantum_cost:^17.1f} | {status}     |")
    
    print("-"*70)
    if all_match:
        print("✅ All eigenvalues match! H_C correctly encodes MaxCut.")
    else:
        print("❌ Mismatch found!")
    
    # Find ground state (maximum cut)
    eigenvalues = np.diag(H)
    max_cut_idx = np.argmax(eigenvalues)
    max_cut_bitstring = format(max_cut_idx, f'0{n}b')
    
    print(f"\n🏆 Maximum cut: {int(max(eigenvalues))} edges")
    print(f"   Solution bitstring: {max_cut_bitstring}")
    
    return H

# Demonstrate on triangle graph
triangle = graphs['triangle']
H_triangle = demonstrate_ising_mapping(triangle)

### 5.5.6 Summary: Why QAOA Works

| Component | Formula | Role in Algorithm |
|-----------|---------|-------------------|
| **Initial State** | $\|+\rangle^{\otimes n}$ | Ground state of $H_M$; equal superposition over all cuts |
| **Cost Unitary** | $e^{-i\gamma H_C}$ | Imprints phase proportional to cut value on each bitstring |
| **Mixer Unitary** | $e^{-i\beta H_M} = \prod_i R_X(2\beta)$ | Creates interference; moves amplitude toward better solutions |
| **ZZ Gate** | CNOT-$R_Z(\gamma)$-CNOT | Implements edge term; gives phase based on same/different partition |
| **Optimization** | Maximize $\langle\psi(\gamma,\beta)|H_C|\psi(\gamma,\beta)\rangle$ | Find parameters that concentrate probability on high-cut states |

**The "Adiabatic-Inspired Alternation" Rule**:
> "Start in the mixer ground state. Alternate between cost evolution (encoding problem) and mixer evolution (enabling transitions). With good parameters, the final state has high probability on optimal or near-optimal solutions."

**Key Insights from the Lectures**:

1. **Why Ising?** The mapping $x_i \to Z_i$ converts classical optimization to quantum eigenvalue problem
2. **Why ZZ?** The edge constraint "different partitions" maps to ZZ eigenvalue -1 (cut) vs +1 (not cut)  
3. **Why alternate?** Mimics Trotterized adiabatic evolution; more layers (higher p) → better approximation
4. **Why p=1 works?** For many graphs, analytical optimal angles exist; provides guaranteed approximation ratio

---

## Section 6: Complete QAOA Circuit

In [ ]:
def create_qaoa_circuit(
    graph: nx.Graph,
    gammas: list,
    betas: list,
    measure: bool = True
) -> QuantumCircuit:
    """
    Create complete QAOA circuit with p layers.
    
    Parameters:
        graph: NetworkX graph defining the problem
        gammas: List of p gamma values (cost evolution angles)
        betas: List of p beta values (mixer evolution angles)
        measure: Whether to add measurement
    
    Returns:
        QuantumCircuit: Complete QAOA circuit
    """
    n_qubits = graph.number_of_nodes()
    p = len(gammas)
    
    assert len(betas) == p, "Number of gammas and betas must match"
    
    qc = QuantumCircuit(n_qubits, n_qubits if measure else 0)
    
    # Initial state: uniform superposition
    qc.h(range(n_qubits))
    qc.barrier()
    
    # Apply p QAOA layers
    for layer in range(p):
        qaoa_layer(qc, graph, gammas[layer], betas[layer])
        if layer < p - 1:
            qc.barrier()
    
    # Measurement
    if measure:
        qc.barrier()
        qc.measure(range(n_qubits), range(n_qubits))
    
    return qc

# Create and display p=1 QAOA circuit
square = graphs['square']
qaoa_p1 = create_qaoa_circuit(square, gammas=[0.5], betas=[0.3])

print("QAOA Circuit (p=1) for Square Graph:")
print(qaoa_p1.draw(output='text', fold=100))

print(f"\nCircuit Statistics:")
print(f"  Depth: {qaoa_p1.depth()}")
print(f"  Gates: {qaoa_p1.count_ops()}")

---

## Section 7: Cost Function Evaluation

In [ ]:
def count_edges_cut(bitstring: str, graph: nx.Graph) -> int:
    """
    Count the number of edges cut by a partition.
    
    Parameters:
        bitstring: Binary string representing partition (Qiskit ordering)
        graph: NetworkX graph
    
    Returns:
        int: Number of edges cut
    """
    edges_cut = 0
    for (i, j) in graph.edges():
        # Qiskit uses reversed ordering: qubit 0 is rightmost
        bit_i = int(bitstring[-(i + 1)])
        bit_j = int(bitstring[-(j + 1)])
        if bit_i != bit_j:  # Edge is cut if endpoints in different partitions
            edges_cut += 1
    return edges_cut

def compute_maxcut_expectation(counts: dict, graph: nx.Graph) -> float:
    """
    Compute expected MaxCut value from measurement results.
    
    Parameters:
        counts: Dictionary of measurement results
        graph: NetworkX graph
    
    Returns:
        float: Expected number of edges cut
    """
    total_cost = 0
    total_shots = sum(counts.values())
    
    for bitstring, count in counts.items():
        edges_cut = count_edges_cut(bitstring, graph)
        total_cost += edges_cut * count
    
    return total_cost / total_shots

def find_best_solution(counts: dict, graph: nx.Graph) -> tuple:
    """
    Find the best solution from measurement results.
    
    Returns:
        tuple: (best_bitstring, max_edges_cut, frequency)
    """
    best_cut = -1
    best_bitstring = None
    best_count = 0
    
    for bitstring, count in counts.items():
        edges_cut = count_edges_cut(bitstring, graph)
        if edges_cut > best_cut:
            best_cut = edges_cut
            best_bitstring = bitstring
            best_count = count
    
    return best_bitstring, best_cut, best_count / sum(counts.values())

# Test on square graph
# Transpile circuit for fake backend
transpiled_qc = transpile(qaoa_p1, backend=fake_backend, optimization_level=3)

# Run with SamplerV2
job = sampler.run([transpiled_qc], shots=1024)
result = job.result()
counts = result[0].data.c.get_counts()

print("Measurement Results (Square Graph, p=1):")
print(f"  Counts: {dict(sorted(counts.items(), key=lambda x: -x[1])[:5])}...")

exp_value = compute_maxcut_expectation(counts, square)
best_sol, best_cut, freq = find_best_solution(counts, square)


print(f"\n📊 Results:")
print(f"  Optimal for square: 4 edges (achieved by 0101 or 1010)")

print(f"  Expected cuts: {exp_value:.3f}")
print(f"  Best solution: {best_sol} with {best_cut} edges cut (found {freq*100:.1f}% of time)")

---

## Section 8: Visualization of Solutions

In [ ]:
def visualize_maxcut_solution(graph: nx.Graph, bitstring: str, ax=None):
    """
    Visualize a MaxCut solution on a graph.
    
    Parameters:
        graph: NetworkX graph
        bitstring: Binary partition string
        ax: Matplotlib axis (optional)
    """
    if ax is None:
        fig, ax = plt.subplots(figsize=(8, 6))
    
    pos = nx.spring_layout(graph, seed=42)
    
    # Color nodes by partition
    colors = []
    for i in range(graph.number_of_nodes()):
        bit = int(bitstring[-(i + 1)])  # Qiskit ordering
        colors.append('lightblue' if bit == 0 else 'lightcoral')
    
    # Color edges by cut status
    edge_colors = []
    edge_widths = []
    for (i, j) in graph.edges():
        bit_i = int(bitstring[-(i + 1)])
        bit_j = int(bitstring[-(j + 1)])
        if bit_i != bit_j:
            edge_colors.append('red')
            edge_widths.append(3)
        else:
            edge_colors.append('gray')
            edge_widths.append(1)
    
    nx.draw(graph, pos, ax=ax, with_labels=True,
            node_color=colors, node_size=700,
            edge_color=edge_colors, width=edge_widths,
            font_size=14, font_weight='bold')
    
    edges_cut = count_edges_cut(bitstring, graph)
    ax.set_title(f"Solution: {bitstring}\nEdges cut: {edges_cut} (red lines)")

# Visualize best solution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Best QAOA solution
visualize_maxcut_solution(square, best_sol, axes[0])
axes[0].set_title(f"QAOA Solution: {best_sol}\nEdges cut: {best_cut}")

# Optimal solution for comparison
visualize_maxcut_solution(square, '0101', axes[1])
axes[1].set_title(f"Optimal Solution: 0101\nEdges cut: 4")

plt.tight_layout()
plt.show()

---

## Section 9: Parameter Optimization

In [ ]:
def qaoa_objective(
    params: np.ndarray,
    graph: nx.Graph,
    p: int,
    shots: int = 1024
) -> float:
    """
    Objective function for QAOA optimization.
    
    We minimize the negative of MaxCut value (to maximize cuts).
    
    Parameters:
        params: Array of [gamma_1, ..., gamma_p, beta_1, ..., beta_p]
        graph: NetworkX graph
        p: QAOA depth
        shots: Number of measurement shots
    
    Returns:
        float: Negative expected MaxCut value
    """
    gammas = params[:p]
    betas = params[p:]
    
    qc = create_qaoa_circuit(graph, gammas, betas)
    
    # Transpile and run with SamplerV2
    transpiled_qc = transpile(qc, backend=fake_backend, optimization_level=3)
    job = sampler.run([transpiled_qc], shots=shots)
    result = job.result()
    counts = result[0].data.c.get_counts()
    
    expectation = compute_maxcut_expectation(counts, graph)
    
    return -expectation  # Negative because we minimize

def run_qaoa_optimization(
    graph: nx.Graph,
    p: int = 1,
    shots: int = 1024,
    maxiter: int = 100,
    verbose: bool = True
) -> dict:
    """
    Run complete QAOA optimization.
    
    Parameters:
        graph: NetworkX graph
        p: QAOA depth (number of layers)
        shots: Shots per evaluation
        maxiter: Maximum optimizer iterations
        verbose: Print progress
    
    Returns:
        dict: Optimization results
    """
    # Random initialization in [0, π]
    np.random.seed(42)  # For reproducibility
    initial_params = np.random.uniform(0, np.pi, 2 * p)
    
    # Track optimization history
    history = {'params': [], 'values': []}
    
    def callback(params):
        val = qaoa_objective(params, graph, p, shots)
        history['params'].append(params.copy())
        history['values'].append(-val)
        if verbose and len(history['values']) % 10 == 0:
            print(f"  Iteration {len(history['values'])}: MaxCut = {-val:.3f}")
    
    if verbose:
        print(f"Starting QAOA optimization (p={p})...")
        print(f"  Graph: {graph.number_of_nodes()} nodes, {graph.number_of_edges()} edges")
    
    # Optimize using COBYLA
    result = minimize(
        qaoa_objective,
        initial_params,
        args=(graph, p, shots),
        method='COBYLA',
        callback=callback,
        options={'maxiter': maxiter}
    )
    
    # Get final results
    optimal_gammas = result.x[:p]
    optimal_betas = result.x[p:]
    optimal_expectation = -result.fun
    
    # Create final circuit with optimal parameters
    final_qc = create_qaoa_circuit(graph, optimal_gammas, optimal_betas)
    
    transpiled_final = transpile(final_qc, backend=fake_backend, optimization_level=3)
    job = sampler.run([transpiled_final], shots=4096)
    final_result = job.result()
    final_counts = final_result[0].data.c.get_counts()
    best_sol, best_cut, freq = find_best_solution(final_counts, graph)
    
    return {
        'optimal_gammas': optimal_gammas,
        'optimal_betas': optimal_betas,
        'expected_cuts': optimal_expectation,
        'best_solution': best_sol,
        'best_cut': best_cut,
        'best_frequency': freq,
        'final_counts': final_counts,
        'history': history,
        'scipy_result': result
    }

# Run optimization on K4 (complete graph)
K4 = graphs['K4']
qaoa_result = run_qaoa_optimization(K4, p=1, shots=1024, maxiter=50)

print(f"\n✅ Optimization Complete!")
print(f"  Optimal γ: {qaoa_result['optimal_gammas']}")
print(f"  Optimal β: {qaoa_result['optimal_betas']}")
print(f"  Expected cuts: {qaoa_result['expected_cuts']:.3f}")


print(f"  Best solution: {qaoa_result['best_solution']} ({qaoa_result['best_cut']} cuts)")
print(f"  Success frequency: {qaoa_result['best_frequency']*100:.1f}%")


---

## Section 10: Depth vs Quality Analysis

In [ ]:
def analyze_depth_scaling(graph: nx.Graph, max_p: int = 3, shots: int = 512):
    """
    Analyze how QAOA performance improves with depth.
    
    Parameters:
        graph: NetworkX graph
        max_p: Maximum depth to test
        shots: Shots per evaluation
    
    Returns:
        dict: Results for each depth
    """
    results = {}
    
    for p in range(1, max_p + 1):
        print(f"\nOptimizing p={p}...")
        result = run_qaoa_optimization(graph, p=p, shots=shots, maxiter=50, verbose=False)
        results[p] = result
        print(f"  Expected cuts: {result['expected_cuts']:.3f}, Best: {result['best_cut']}")
    
    return results

# Analyze depth scaling on butterfly graph
butterfly = graphs['butterfly']
print(f"Analyzing depth scaling on butterfly graph...")
print(f"  (6 edges, optimal MaxCut = 5)")

depth_results = analyze_depth_scaling(butterfly, max_p=3, shots=512)

# Plot results
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: Expected cuts vs depth
depths = list(depth_results.keys())
expected = [depth_results[p]['expected_cuts'] for p in depths]
best_cuts = [depth_results[p]['best_cut'] for p in depths]

axes[0].bar([p - 0.15 for p in depths], expected, 0.3, label='Expected', color='steelblue')
axes[0].bar([p + 0.15 for p in depths], best_cuts, 0.3, label='Best Found', color='coral')
axes[0].axhline(y=5, color='green', linestyle='--', label='Optimal (5)')
axes[0].set_xlabel('QAOA Depth (p)')
axes[0].set_ylabel('Edges Cut')
axes[0].set_title('QAOA Performance vs Depth')
axes[0].legend()
axes[0].set_xticks(depths)

# Right: Optimization convergence for p=2
if 2 in depth_results and depth_results[2]['history']['values']:
    axes[1].plot(depth_results[2]['history']['values'], 'b-', linewidth=2)
    axes[1].set_xlabel('Optimization Iteration')
    axes[1].set_ylabel('Expected Cuts')
    axes[1].set_title('Optimization Convergence (p=2)')
    axes[1].axhline(y=5, color='green', linestyle='--', label='Optimal')
    axes[1].legend()

plt.tight_layout()
plt.show()

print("\n📊 Summary:")
for p in depths:
    approx_ratio = depth_results[p]['expected_cuts'] / 5  # Optimal is 5
    print(f"  p={p}: Expected = {depth_results[p]['expected_cuts']:.2f}, "
          f"Approximation ratio = {approx_ratio:.2%}")

---

## Section 11: Trap Demonstrations

In [ ]:
print("🚨 TRAP 1: Wrong Bitstring Indexing")
print("="*50)

# Demonstrate Qiskit's reversed bit ordering
test_graph = nx.Graph()
test_graph.add_edges_from([(0, 1), (1, 2)])

bitstring = "110"  # In Qiskit: qubit 2 = 1, qubit 1 = 1, qubit 0 = 0

# WRONG way
def count_edges_cut_WRONG(bitstring, graph):
    edges_cut = 0
    for (i, j) in graph.edges():
        bit_i = int(bitstring[i])  # ❌ Wrong!
        bit_j = int(bitstring[j])
        if bit_i != bit_j:
            edges_cut += 1
    return edges_cut

# CORRECT way
def count_edges_cut_CORRECT(bitstring, graph):
    edges_cut = 0
    for (i, j) in graph.edges():
        bit_i = int(bitstring[-(i+1)])  # ✅ Correct!
        bit_j = int(bitstring[-(j+1)])
        if bit_i != bit_j:
            edges_cut += 1
    return edges_cut

print(f"Bitstring: '{bitstring}'")
print(f"Graph edges: {list(test_graph.edges())}")
print(f"\n❌ Wrong indexing result: {count_edges_cut_WRONG(bitstring, test_graph)} edges cut")
print(f"✅ Correct indexing result: {count_edges_cut_CORRECT(bitstring, test_graph)} edges cut")
print(f"\nExplanation:")
print(f"  Qiskit ordering: bitstring[0] = qubit {len(bitstring)-1}, bitstring[-1] = qubit 0")
print(f"  For '110': qubit 0 = 0, qubit 1 = 1, qubit 2 = 1")

print("\n" + "="*50)
print("🚨 TRAP 2: Bad Parameter Initialization")
print("="*50)

# Compare different initializations
np.random.seed(123)
square = graphs['square']

# Zero initialization (trivial)
qc_zero = create_qaoa_circuit(square, gammas=[0], betas=[0])
transpiled_zero = transpile(qc_zero, backend=fake_backend, optimization_level=3)
job_zero = sampler.run([transpiled_zero], shots=1000)
result_zero = job_zero.result()
exp_zero = compute_maxcut_expectation(result_zero[0].data.c.get_counts(), square)

# Good initialization
qc_good = create_qaoa_circuit(square, gammas=[np.pi/4], betas=[np.pi/8])
transpiled_good = transpile(qc_good, backend=fake_backend, optimization_level=3)
job_good = sampler.run([transpiled_good], shots=1000)
result_good = job_good.result()
exp_good = compute_maxcut_expectation(result_good[0].data.c.get_counts(), square)

print(f"\nLesson: γ=0 means no cost evolution → uniform output (random guessing)")

print(f"Zero initialization (γ=0, β=0): {exp_zero:.3f} edges cut")
print(f"Good initialization (γ=π/4, β=π/8): {exp_good:.3f} edges cut")

---

## Section 12: Exercises

### 🟢 Exercise 1: Implement QAOA for a Different Graph

Create a 6-node cycle graph and run QAOA optimization.

In [ ]:
# TODO: Create a 6-node cycle graph
# cycle6 = nx.cycle_graph(6)

# TODO: Run QAOA with p=2
# result = run_qaoa_optimization(cycle6, p=2, shots=1024, maxiter=50)

# TODO: Visualize the best solution
# visualize_maxcut_solution(cycle6, result['best_solution'])

# YOUR CODE HERE

### 🟡 Exercise 2: Implement Weighted MaxCut

Extend the code to handle weighted edges where the cost function is:
$$C(x) = \sum_{(i,j) \in E} w_{ij} \cdot \mathbb{1}[x_i \neq x_j]$$

In [ ]:
# TODO: Create weighted graph
# weighted_graph = nx.Graph()
# weighted_graph.add_edge(0, 1, weight=2.0)
# weighted_graph.add_edge(1, 2, weight=1.0)
# weighted_graph.add_edge(0, 2, weight=3.0)

# TODO: Modify zz_gate and cost_layer to use edge weights
# def weighted_cost_layer(qc, graph, gamma):
#     for (i, j, data) in graph.edges(data=True):
#         weight = data.get('weight', 1.0)
#         zz_gate(qc, i, j, gamma * weight)

# YOUR CODE HERE

### 🔴 Exercise 3: Parameter Landscape Visualization

For p=1 QAOA on a triangle graph, create a 2D heatmap of the cost function over the (γ, β) parameter space.

In [ ]:
# TODO: Create meshgrid of gamma and beta values
# gammas = np.linspace(0, np.pi, 20)
# betas = np.linspace(0, np.pi/2, 20)
# Gamma, Beta = np.meshgrid(gammas, betas)

# TODO: Evaluate cost at each point
# costs = np.zeros_like(Gamma)
# for i in range(len(gammas)):
#     for j in range(len(betas)):
#         costs[j, i] = -qaoa_objective([gammas[i], betas[j]], triangle, 1, 256)

# TODO: Create heatmap
# plt.contourf(Gamma, Beta, costs, levels=20)
# plt.colorbar(label='Expected Cuts')
# plt.xlabel('γ')
# plt.ylabel('β')

# YOUR CODE HERE

---

## Section 13: Quick Checks ✅

In [ ]:
print("🧠 Quick Check Questions")
print("="*50)

questions = [
    {
        'q': "1. What gates implement the mixer Hamiltonian exp(-iβΣXi)?",
        'a': "Rx(2β) gates on each qubit - the mixer factorizes into single-qubit rotations"
    },
    {
        'q': "2. How many ZZ gates are needed for MaxCut on K5 (complete graph, 5 nodes)?",
        'a': "C(5,2) = 10 ZZ gates (one per edge)"
    },
    {
        'q': "3. Why do we start with Hadamard gates in QAOA?",
        'a': "To create uniform superposition over all possible solutions - this is the ground state of the mixer Hamiltonian"
    },
    {
        'q': "4. What's the relationship between QAOA and adiabatic quantum computing?",
        'a': "QAOA is a discretized/Trotterized version - adiabatic QC uses continuous evolution while QAOA uses discrete layers with optimizable parameters"
    }
]

for q_dict in questions:
    print(f"\n{q_dict['q']}")
    input("Press Enter to see answer...")
    print(f"Answer: {q_dict['a']}")


---

## Section 14: Summary and Next Steps

### Key Takeaways

1. **QAOA Structure**: Initial superposition → (Cost layer → Mixer layer) × p → Measure

2. **ZZ Gate**: CNOT-Rz(γ)-CNOT implements ZZ evolution for MaxCut edges

3. **Cost Evaluation**: Count edges cut from bitstrings, weight by measurement frequency

4. **Optimization**: Classical optimizer updates (γ, β) to maximize expected cuts

5. **Depth Tradeoff**: Higher p → better solutions but deeper circuits

### What You've Implemented

- ✅ ZZ gate construction
- ✅ Complete QAOA circuit builder
- ✅ MaxCut cost function evaluation
- ✅ Classical parameter optimization
- ✅ Solution visualization

### Next Steps

- **Module 11**: HHL Algorithm for linear systems
- **Extension**: Warm-start QAOA using classical approximations
- **Hardware**: Run on IBM Quantum with error mitigation
- **Applications**: Portfolio optimization, vehicle routing